## 3. 베이스라인 모델
- LightGBM을 사용함.
- 피처엔지니어링 : 핵심
    1. 피처명 한글화
    2. 다운 캐스팅
    3. 데이터 조합 생성
    4. 파생 피처 추가
    5. 테스트 데이터 이어붙이기
    6. 나머지 데이터 병합 (최종 데이터 생성)
    7. 훈련, 검증, 테스트 데이터 만들기


### 데이터 불러오기
LighGBM으로 범주형 데이터를 모델링하면 불필요한 경고문구가 뜸.

In [3]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings(action='ignore') # 경고문구 생략


# 데이터 경로
data_path='./data/'

sales_train = pd.read_csv(data_path + 'sales_train.csv')
shops = pd.read_csv(data_path + 'shops.csv')
items = pd.read_csv(data_path + 'items.csv')
item_categories = pd.read_csv(data_path + 'item_categories.csv')
test = pd.read_csv(data_path + 'test.csv')
submission = pd.read_csv(data_path + 'sample_submission.csv')

### 1-1. 피처 엔지니어링 1 : 피처명 한글화
- 피처명이 영어면 헷갈릴 수 있음.
- 성능 개선 절에서 파생 피처 만들 때 피처를 쉽게 알아볼 수 있음.

In [22]:
sales_train = sales_train.rename(columns={'date': '날짜', 
                                          'date_block_num': '월ID',
                                          'shop_id': '상점ID',
                                          'item_id': '상품ID',
                                          'item_price': '판매가',
                                          'item_cnt_day': '판매량'})

sales_train.head()

,날짜,월ID,상점ID,상품ID,판매가,판매량
0,02.01.2013,0,59,22154,999.00,1
1,03.01.2013,0,25,2552,899.00,1
2,05.01.2013,0,25,2552,899.00,-1
3,06.01.2013,0,25,2554,1709.05,1
4,15.01.2013,0,25,2555,1099.00,1


In [23]:
shops = shops.rename(columns={'shop_name': '상점명',
                              'shop_id': '상점ID'})

shops.head()

,상점명,상점ID
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [24]:
items = items.rename(columns={'item_name': '상품명',
                              'item_id': '상품ID',
                              'item_category_id': '상품분류ID'})

items.head()

,상품명,상품ID,상품분류ID
0,! ВО ВЛАСТИ НАВАЖДЕНИЯ (ПЛАСТ.) D,0,40
1,!ABBYY FineReader 12 Professional Edition Full...,1,76
2,***В ЛУЧАХ СЛАВЫ (UNV) D,2,40
3,***ГОЛУБАЯ ВОЛНА (Univ) D,3,40
4,***КОРОБКА (СТЕКЛО) D,4,40


In [27]:
item_categories = item_categories.rename(columns={'item_category_name': '상품분류명',
                                                  'item_category_id': '상품분류ID'})

item_categories.head()

,상품분류명,상품분류ID
0,PC - Гарнитуры/Наушники,0
1,Аксессуары - PS2,1
2,Аксессуары - PS3,2
3,Аксессуары - PS4,3
4,Аксессуары - PSP,4


In [28]:
test = test.rename(columns={'shop_id': '상점ID',
                            'item_id': '상품ID'})

test.head()

,ID,상점ID,상품ID
0,0,5,5037
1,1,5,5320
2,2,5,5233
3,3,5,5232
4,4,5,5268


### 1-2. 피처엔지니어링 2 : 데이터 다운캐스팅
- 다운캐스팅 : 더 작은 데이터 타입으로 변환하는 작업
    - 데이터가 작은데 큰 데이터 타입을 사용하면 메모리를 낭비하게 됨. 주어진 데이터 크기에 맞는 타입을 사용해야함.
    - pandas는 기본적으로 정수형은 int64, 실수형은 float64로 할당함.
    - 모든 피처가 최대 타입을 사용할 이유는 없으니, 더 작은 타입으로 할당해야함. int8, int16, int32, float16, float32
    - 메모리 낭비를 막을 뿐 아니라 훈련 속도도 빨라짐.

In [14]:
def downcast(df, verbose=True):
    start_mem = df.memory_usage().sum() / 1024**2
    for col in df.columns:
        dtype_name = df[col].dtype.name
        if dtype_name == 'object':
            pass
        elif dtype_name == 'bool':
            df[col] = df[col].astype('int8')
        elif dtype_name.startswith('int') or (df[col].round() == df[col]).all():
            df[col] = pd.to_numeric(df[col], downcast='integer')
        else:
            df[col] = pd.to_numeric(df[col], downcast='float')
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose:
        print('{:.1f}% 압축됨'.format(100 * (start_mem - end_mem)/ start_mem))
        
    return df

In [15]:
all_df = [sales_train, shops, items, item_categories, test]
for df in all_df:
    df = downcast(df)

54.2% 압축됨
38.6% 압축됨
54.2% 압축됨
39.9% 압축됨
70.8% 압축됨


### 1-3. 피처 엔지니어링 3 : 데이터 조합 생성
- 테스트 데이터의 피처는 ID를 제외하면 상점ID, 상품ID 피처이다.
- 우리가 예측해야하는 값은 상점의 상품별 월간 판매량이다.
- 따라서, 월, 상점, 상품별 조합이 필요하다. -> 월ID, 상점ID, 상품ID 피처 조합이 필요.
- 이때, 상품 팔린게 없어서 데이터가 없기 보다는 판매량이 0이더라도 데이터가 있는게 낫다.

-> 데이터 조합은 itertools가 제공하는 product()함수로 쉽게 만들 수 있다.

In [40]:
from itertools import product

train = []
# 월ID, 상점ID, 상품ID 조합 생성 
for i in sales_train['월ID'].unique():
    all_shop = sales_train.loc[sales_train['월ID']==i, '상점ID'].unique()
    all_item = sales_train.loc[sales_train['월ID']==i, '상품ID'].unique()
    train.append(np.array(list(product([i], all_shop, all_item))))


idx_features = ['월ID', '상점ID', '상품ID'] # 기준 피처
# 리스트 타입인 train을 DataFrame 타입으로 변환 : 34개의 배열을 하나로 합쳐서줌.
train = pd.DataFrame(np.vstack(train), columns=idx_features)


train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


### 1-4. 피처 엔지니어링 4 : 타깃값(월간 판매량) 추가
- train에 다른 데이터를 추가할 것임.
- 타깃값인 상점의 상품별 월간 판매량임.
- sales_train에 판매량 피처가 있지만, 월간 판매량이 필요함. 따라서 groupby()를 활용한다.

In [38]:
# idx_features를 기준으로 그룹화해 판매량 합 구하기 
group = sales_train.groupby(idx_features).agg({'판매량': 'sum'})

# 인덱스 재설정
group = group.reset_index()

# 피처명을 '판매량'에서 '월간 판매량'으로 변경
group = group.rename(columns={'판매량': '월간 판매량'})

group

,월ID,상점ID,상품ID,월간 판매량
0,0,0,32,6
1,0,0,33,3
2,0,0,35,1
3,0,0,43,1
4,0,0,51,2
...,...,...,...,...
1609119,33,59,22087,6
1609120,33,59,22088,2
1609121,33,59,22091,1
1609122,33,59,22100,1


In [41]:
train

,월ID,상점ID,상품ID
0,0,59,22154
1,0,59,2552
2,0,59,2554
3,0,59,2555
4,0,59,2564
...,...,...,...
10913845,33,21,7635
10913846,33,21,7638
10913847,33,21,7640
10913848,33,21,7632


In [42]:
# train과 group 병합하기
train = train.merge(group, on=idx_features, how='left')

train

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
10913845,33,21,7635,NaN
10913846,33,21,7638,NaN
10913847,33,21,7640,NaN
10913848,33,21,7632,NaN


- 결측값이 많이 보이는데, 모든 조합을 만들어넣고, 특정 조합만 넣어서 그렇다.
- 어차피 없는 조합은 월간 판매량이 0이므로, 결측값을 0으로 조정한다.

## 가비지 컬렉션
- group 데이터는 더 이상 필요 없으니 메모리 절약 차원에서 가비지 컬렉션 해준다.
- 가비지 컬렉션 : 쓰레기 수거로, 할당한 메모리 중 더이상 사용하지 않는 영역을 해제하는 기능.

In [43]:
import gc # 가비지 컬렉터 불러오기

del group # 더는 사용하지 않는 변수 지정
gc.collect(); # 가비지 컬렉션 수행

### 1-5. 피처 엔지니어링5 : 테스트 데이터 이어붙이기
- 월ID, 상점ID, 상품ID + 상점의 상품별 월간 판매량(타깃값)을 추가했음.
- 테스트 데이터를 이어붙이자. (이어붙인다 = 테이블을 위아래로 합치기, concat() , 병합 = 좌우로 합치기, merge())
- 이어 붙이기 전에 test data에 월ID 피처를 추가해야함.
- 테스트는 2015년 11월 판매기록 이므로, 34로 지정함.

In [44]:
test['월ID'] = 34

In [45]:
test

,ID,상점ID,상품ID,월ID
0,0,5,5037,34
1,1,5,5320,34
2,2,5,5233,34
3,3,5,5232,34
4,4,5,5268,34
...,...,...,...,...
214195,214195,45,18454,34
214196,214196,45,16188,34
214197,214197,45,15757,34
214198,214198,45,19648,34


- test는 ID 피처도 갖고 있는데, 필요없으므로, ID를 제거하고, test를 이어붙인다.

In [46]:
# train과 test 이어붙이기
all_data = pd.concat([train, test.drop('ID', axis=1)],
                     ignore_index=True, # 기존 인덱스 무시(0부터 새로 시작)
                     keys=idx_features) # 이어붙이는 기준이 되는 피처

In [47]:
all_data

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,NaN
2,0,59,2554,NaN
3,0,59,2555,NaN
4,0,59,2564,NaN
...,...,...,...,...
11128045,34,45,18454,NaN
11128046,34,45,16188,NaN
11128047,34,45,15757,NaN
11128048,34,45,19648,NaN


- test와 train이 모두 합쳐진 데이터로, 결측값을 전부 0으로 만든다.

In [48]:
# 결측값을 0으로 대체
all_data = all_data.fillna(0)

all_data

,월ID,상점ID,상품ID,월간 판매량
0,0,59,22154,1.0
1,0,59,2552,0.0
2,0,59,2554,0.0
3,0,59,2555,0.0
4,0,59,2564,0.0
...,...,...,...,...
11128045,34,45,18454,0.0
11128046,34,45,16188,0.0
11128047,34,45,15757,0.0
11128048,34,45,19648,0.0


### 1-6. 피처 엔지니어링 6 : 나머지 데이터 병합 (최종 데이터 생성)
- 추가 정보로 제공한 shops, items, item_categories 데이터를 all_data에 병합할 것임.
- 병합은 좌우 증가이므로 merge 사용.
- 메모리 절약을 위해 데이터 다운 캐스팅과 가비지 컬렉션을 수행함.

In [49]:
# 나머지 데이터 병합
all_data = all_data.merge(shops, on='상점ID', how='left')
all_data = all_data.merge(items, on='상품ID', how='left')
all_data = all_data.merge(item_categories, on='상품분류ID', how='left')

# 데이터 다운캐스팅
all_data = downcast(all_data)

# 가비지 컬렉션
del shops, items, item_categories
gc.collect();

26.4% 압축됨


모든게 완벽하게 이루어졌고, all_data를 본다.

In [50]:
all_data.head()

,월ID,상점ID,상품ID,월간 판매량,상점명,상품명,상품분류ID,상품분류명
0,0,59,22154,1,"Ярославль ТЦ ""Альтаир""",ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray
1,0,59,2552,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил
2,0,59,2554,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил
3,0,59,2555,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства
4,0,59,2564,0,"Ярославль ТЦ ""Альтаир""",DEEP PURPLE Perihelion: Live In Concert DVD (К...,59,Музыка - Музыкальное видео


이전에 말했듯이 ID와 이름이 일대일 대응이다.
- 상점명
- 상품명
- 상품분류명

이거 모두 id랑 동일해서 제거한다.

In [51]:
all_data = all_data.drop(['상점명', '상품명', '상품분류명'], axis=1)

### 1-7. 피처 엔지니어링 7 : 마무리
- 앞에서 모든 데이터를 병합해서 all_data를 만들었음.
- 이걸 훈련, 검증, 테스트용 데이터로 나누겠음.


- 훈련 데이터 : 2013.01~2015.09 까지 판매내역 (0 ~ 32)
- 검증 데이터 : 2015.10 판매내역 (33)
- 테스트 데이터 : 2015.11 판매내역 (34)

In [52]:
# 훈련 데이터 (피처)
X_train = all_data[all_data['월ID'] < 33]
X_train = X_train.drop(['월간 판매량'], axis=1)

# 검증 데이터 (피처)
X_valid = all_data[all_data['월ID'] == 33]
X_valid = X_valid.drop(['월간 판매량'], axis=1)

# 테스트 데이터 (피처)
X_test = all_data[all_data['월ID'] == 34]
X_test = X_test.drop(['월간 판매량'], axis=1)

# 훈련 데이터 (타깃값)
y_train = all_data[all_data['월ID'] < 33]['월간 판매량']
y_train = y_train.clip(0, 20) # 타깃값을 0 ~ 20로 제한

# 검증 데이터 (타깃값)
y_valid = all_data[all_data['월ID'] == 33]['월간 판매량']
y_valid = y_valid.clip(0, 20)

# clip() : 하한값, 상한값을 넘을 경우, 이 값으로 맞춰주는 역할. clipping이라 함.
# 하한값보다 작은건 하한값으로, 상한값 보다 큰건 상한값으로 맞춰줌.

In [53]:
# 가비지 콜렉터 : 필요없는 데이터 제거
del all_data
gc.collect();

이로써 모델링에 필요한 모든 데이터를 완성했음.

### 2. 모델 훈련 및 성능 검증
- 베이스 라인 : LightGBM
- 기본 파라미터만 설정하고, LightGBM용 데이터셋을 만들어서 훈련할 것임.
- categorical_feature 파라미터에 범주형 데이터를 전달하면 된다.


- 범주형 데이터 : 상점ID, 상품분류ID, 상품ID(이건 수치형으로 빼겠음)
- 수치형 데이터 : 상품ID : 상품 ID는 고유값 개수가 많음. LightGBM의 경우 고윳값 개수가 많은 범주형 데이터는 수치형 데이터로 취급해야 성능이 더 잘나온다.
> 범주형 데이터는 고윳값 하나하나가 일정한 의미를 갖는데, 너무 많으면, 의미가 상쇄되어 수치형 데이터와 다를게 없어지므로.

In [55]:
import lightgbm as lgb

# LightGBM 하이퍼파라미터
params = {'metric': 'rmse', # 평가지표 = rmse
          'num_leaves': 255,
          'learning_rate': 0.01,
          'force_col_wise': True,
          'random_state': 10}

# 범주형 피처 설정
cat_features = ['상점ID', '상품분류ID']

# c.f. 다음과 같이 category type으로 바꿔주면 LighGBM 모델에 categorical_feature에 안줘도 됨.
# for cat_feature in cat_features:
#     all_data[cat_feature] = all_data[cat_feature].astype('category')

# LightGBM 훈련 및 검증 데이터셋
dtrain = lgb.Dataset(X_train, y_train)
dvalid = lgb.Dataset(X_valid, y_valid)

# LightGBM 모델 훈련
lgb_model = lgb.train(params=params,
                      train_set=dtrain,
                      num_boost_round=500,
                      valid_sets=(dtrain, dvalid),
                      categorical_feature=cat_features,
                      verbose_eval=50)  

[LightGBM] [Info] Total Bins 426
[LightGBM] [Info] Number of data points in the train set: 10675678, number of used features: 4
[LightGBM] [Info] Start training from score 0.299125
[50]	training's rmse: 1.14777	valid_1's rmse: 1.06755
[100]	training's rmse: 1.11425	valid_1's rmse: 1.0386
[150]	training's rmse: 1.09673	valid_1's rmse: 1.02671
[200]	training's rmse: 1.08573	valid_1's rmse: 1.02027
[250]	training's rmse: 1.07722	valid_1's rmse: 1.01661
[300]	training's rmse: 1.0698	valid_1's rmse: 1.0138
[350]	training's rmse: 1.06317	valid_1's rmse: 1.01084
[400]	training's rmse: 1.05734	valid_1's rmse: 1.00936
[450]	training's rmse: 1.05224	valid_1's rmse: 1.00818
[500]	training's rmse: 1.04792	valid_1's rmse: 1.00722


평균 제곱근 편차(Root Mean Square Deviation; RMSD) 또는 평균 제곱근 오차(Root Mean Square Error; RMSE)는 추정 값 또는 모델이 예측한 값과 실제 환경에서 관찰되는 값의 차이를 다룰 때 흔히 사용하는 측도이다. 0에 가까울수록 좋음

### 3. 예측 및 결과 제출
- 타깃값은 0~20 사이이므로, clip()으로 clipping하여 제공해야한다.

In [56]:
# 예측
preds = lgb_model.predict(X_test).clip(0, 20)

In [58]:
# 제출 파일 생성
submission['item_cnt_month'] = preds
submission.to_csv('submission.csv', index=False)

끝으로, 메모리를 아끼기 위해 가비지 컬렉션을 해준다.

In [59]:
del X_train, y_train, X_valid, y_valid, X_test, lgb_model, dtrain, dvalid
gc.collect();

결과는 퍼플릭 점수만 인정되는데, 1.08534이다.